# 4-4 Feature・Stage モジュールの解説と実装

## Feature モジュールの構成と実装
VGG-19 を利用した Feature モジュールの構成を次に示す．

<img src="../image/p212.png">

Feature モジュールでは10層目の畳み込み層まで，VGG-19 の構成をそのまま用いる．
その後，2層の畳み込み層 ＋ ReLU を経て Feature モジュールの出力とする．

In [6]:
import torch
import torchvision
import torch.nn as nn

class OpenPose_Feature(nn.Module):
    def __init__(self):
        super(OpenPose_Feature, self).__init__()
        
        # VGG-19 の最初 10 個の畳み込みを使用
        # 初めて実行する際は、モデルの重みパラメータをダウンロードするため、実行に時間がかかる
        vgg19 = torchvision.models.vgg19(pretrained=True)
        model = {}
        model["block0"] = vgg19.features[0:22]  # VGG-19 の最初の10層
        
        # 新たに畳み込み層を2つ用意
        model["block0"].add_module("23", torch.nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1))
        model["block0"].add_module("24", torch.nn.ReLU(inplace=True))
        model["block0"].add_module("25", torch.nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1))
        model["block0"].add_module("26", torch.nn.ReLU(inplace=True))
        
        self.model = model["block0"]
        
    def forward(self, x):
        outputs = self.model(x)
        return outputs

## 各 Stage モジュールの block の構成と実装
以下に各 Stage のブロックの構成を示す．

<img src="../image/p214.png">

Stage1 は Feature モジュールからの出力テンソルを受け取り，Stage2〜6 は Feature モジュールの出力と前段の Stage モジュールの出力を結合したテンソルを入力とする．
各 Stage の各 block は全て畳み込み層と ReLU だけで構成されている．
block1 と block2 の最後の出力は，チャネル数が異なるだけでその他の構成は同じである．  
ここでは，nn.Sequential() で作成されたネットワークモデルを生成する関数 make_OpenPose_block 関数を作成する．
この関数では次の4つの手続きを行う．

1. サブネットワークを構成するユニットのコンフィグレーションを設定する  
全 Stage，全 block のコンフィグレーションを用意しておき、与えられた引数の block_name の設定を使用するようにする
1. コンフィグレーションの内容に合わせて畳み込み層と ReLU を生成し，リスト変数 layers に格納
1. layers の中身のユニット情報を使用し nn.Sequential() クラスのネットワークモデル net を作成
1. net 内の畳み込み層の重みを初期化する

In [10]:
from torch.nn import init

def make_OpenPose_block(block_name):
    """
    コンフィグレーション変数から OpenPose の Stage モジュールの blcok を作成
    nn.Module ではなく nn.Sequential にする
    """

    # 1. コンフィグレーションの辞書変数 blocks を作成しネットワークを生成させる
    # 最初に全パターンの辞書を用意し引数 block_name のみを生成する
    blocks = {}
    
    # Stage1
    blocks["block1_1"] = [{"conv5_1_CPM_L1": [128, 128, 3, 1, 1]},
                          {"conv5_2_CPM_L1": [128, 128, 3, 1, 1]},
                          {"conv5_3_CPM_L1": [128, 128, 3, 1, 1]},
                          {"conv5_4_CPM_L1": [128, 512, 1, 1, 0]},
                          {"conv5_5_CPM_L1": [512, 38, 1, 1, 0]},]
    
    blocks["block1_2"] = [{"conv5_1_CPM_L2": [128, 128, 3, 1, 1]},
                          {"conv5_2_CPM_L2": [128, 128, 3, 1, 1]},
                          {"conv5_3_CPM_L2": [128, 128, 3, 1, 1]},
                          {"conv5_4_CPM_L2": [128, 512, 1, 1, 0]},
                          {"conv5_5_CPM_L2": [512, 19, 1, 1, 0]},]
    
    # Stage 2 ~ 6
    for i in range(2, 7):
        blocks['block%d_1' % i] = [{'Mconv1_stage%d_L1' % i: [185, 128, 7, 1, 3]},
                                   {'Mconv2_stage%d_L1' % i: [128, 128, 7, 1, 3]},
                                   {'Mconv3_stage%d_L1' % i: [128, 128, 7, 1, 3]},
                                   {'Mconv4_stage%d_L1' % i: [128, 128, 7, 1, 3]},
                                   {'Mconv5_stage%d_L1' % i: [128, 128, 7, 1, 3]},
                                   {'Mconv6_stage%d_L1' % i: [128, 128, 1, 1, 0]},
                                   {'Mconv7_stage%d_L1' % i: [128, 38, 1, 1, 0]},]
        
        blocks['block%d_2' % i] = [{'Mconv1_stage%d_L2' % i: [185, 128, 7, 1, 3]},
                                   {'Mconv2_stage%d_L2' % i: [128, 128, 7, 1, 3]},
                                   {'Mconv3_stage%d_L2' % i: [128, 128, 7, 1, 3]},
                                   {'Mconv4_stage%d_L2' % i: [128, 128, 7, 1, 3]},
                                   {'Mconv5_stage%d_L2' % i: [128, 128, 7, 1, 3]},
                                   {'Mconv6_stage%d_L2' % i: [128, 128, 1, 1, 0]},
                                   {'Mconv7_stage%d_L2' % i: [128, 19, 1, 1, 0]},]
        
    # 引数 block_name のコンフィグレーション辞書を取り出す
    cfg_dict = blocks[block_name]
    
    # 2. コンフィグレーション内容をリスト変数 layers に格納
    layers = []
    
    # 0番目から最後の層までを作成
    for i in range(len(cfg_dict)):
        for k, v in cfg_dict[i].items():
            if "pool" in k:
                layers += [nn.MaxPool2d(kernel_size=v[0], stride=v[1], padding=v[2])]
            else:
                conv2d = nn.Conv2d(in_channels=v[0], out_channels=v[1], kernel_size=v[2], stride=v[3], padding=v[4])
                layers += [conv2d, nn.ReLU(inplace=True)]
                
    # 3. layers を Sequential にする
    # ただし最後に ReLU はいらないのでその手前までを使用する
    net = nn.Sequential(*layers[:-1])
    
    # 4. 初期化関数の設定し、畳み込み層を初期化する
    def _initialize_weights_norm(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    init.constant_(m.bias, 0.0)
                    
    net.apply(_initialize_weights_norm)
    
    return net

## 動作確認

In [11]:
class OpenPoseNet(nn.Module):
    def __init__(self):
        super(OpenPoseNet, self).__init__()
        
        # Feature モジュール
        self.model0 = OpenPose_Feature()
        
        # Stage モジュール
        # PAFs (Part Affinity Fields) 側
        self.model1_1 = make_OpenPose_block("block1_1")
        self.model2_1 = make_OpenPose_block("block2_1")
        self.model3_1 = make_OpenPose_block("block3_1")
        self.model4_1 = make_OpenPose_block("block4_1")
        self.model5_1 = make_OpenPose_block("block5_1")
        self.model6_1 = make_OpenPose_block("block6_1")
        
        # confidence heatmap 側
        self.model1_2= make_OpenPose_block("block1_2")
        self.model2_2= make_OpenPose_block("block2_2")
        self.model3_2= make_OpenPose_block("block3_2")
        self.model4_2= make_OpenPose_block("block4_2")
        self.model5_2= make_OpenPose_block("block5_2")
        self.model6_2= make_OpenPose_block("block6_2")
        
        
    def forward(self, x):
        ''' 順伝播関数の定義 '''
        
        # Feature モジュール
        out1 = self.model0(x)
        
        # Stage1
        out1_1 = self.model1_1(out1) # PAFs
        out1_2 = self.model1_2(out1) # confidence heatmaps
        
        # Stage2
        out2 = torch.cat([out1_1, out1_2, out1], 1) # チャネルの次元で結合
        out2_1 = self.model2_1(out2) # PAFs
        out2_2 = self.model2_2(out2) # confidence heatmaps
        
        # Stage3
        out3 = torch.cat([out2_1, out2_2, out1], 1) # チャネルの次元で結合
        out3_1 = self.model3_1(out3) # PAFs
        out3_2 = self.model3_2(out3) # confidence heatmaps
        
        # Stage4
        out4 = torch.cat([out3_1, out3_2, out1], 1) # チャネルの次元で結合
        out4_1 = self.model4_1(out4) # PAFs
        out4_2 = self.model4_2(out4) # confidence heatmaps
        
        # Stage5
        out5 = torch.cat([out4_1, out4_2, out1], 1) # チャネルの次元で結合
        out5_1 = self.model5_1(out5) # PAFs
        out5_2 = self.model5_2(out5) # confidence heatmaps
        
        # Stage6
        out6 = torch.cat([out5_1, out5_2, out1], 1) # チャネルの次元で結合
        out6_1 = self.model6_1(out6) # PAFs
        out6_2 = self.model6_2(out6) # confidence heatmaps
        
        # 損失の計算用に各 Stage の結果を格納
        saved_for_loss = []
        saved_for_loss.append(out1_1)
        saved_for_loss.append(out1_2)
        saved_for_loss.append(out2_1)
        saved_for_loss.append(out2_2)
        saved_for_loss.append(out3_1)
        saved_for_loss.append(out3_2)
        saved_for_loss.append(out4_1)
        saved_for_loss.append(out4_2)
        saved_for_loss.append(out5_1)
        saved_for_loss.append(out5_2)
        saved_for_loss.append(out6_1)
        saved_for_loss.append(out6_2)
        
        # 最終的な PAFs の out6_1 と confidence heatmap の out6_2，
        # 損失計算用に各ステージでの PAFs と heatmap を格納した saved_for_loss を出力
        # out6_1:torch.Size([minibatch, 38, 46, 46])
        # out6_2:torch.Size([minibatch, 19, 46, 46])
        # saved_for_loss:[out1_1, out_1_2, ・・・, out6_2]
        
        return (out6_1, out6_2), saved_for_loss

In [12]:
# モデルの定義
net = OpenPoseNet()
net.train()

# ダミーデータの作成
batch_size = 2
dummy_img = torch.rand(batch_size, 3, 368, 368)

# 計算
outputs = net(dummy_img)
print(outputs)

((tensor([[[[-2.7740e-05, -2.0082e-05, -8.8883e-05,  ...,  5.8848e-05,
            3.6739e-05,  3.0854e-05],
          [ 6.8427e-06,  1.2409e-04, -3.1681e-05,  ...,  5.9939e-05,
            1.2626e-04, -2.6380e-05],
          [ 4.8106e-05,  5.8993e-05,  4.3573e-05,  ...,  1.2382e-04,
            1.3005e-04, -1.6012e-05],
          ...,
          [ 1.7219e-04,  1.3696e-04,  1.0846e-04,  ...,  2.5345e-04,
            1.4662e-04, -1.9136e-05],
          [ 9.8239e-05,  1.4655e-04,  2.5685e-04,  ...,  2.6609e-04,
            1.9870e-04,  5.8497e-05],
          [ 1.0828e-04,  1.5393e-04,  1.7267e-04,  ...,  1.5035e-04,
            2.4709e-05,  1.1319e-05]],

         [[-1.3639e-05, -6.1840e-05,  6.8441e-05,  ..., -6.6908e-06,
           -5.0739e-05, -5.2562e-05],
          [-5.1959e-05, -8.7089e-05,  1.2147e-05,  ..., -4.7557e-05,
           -1.2161e-04, -1.0595e-04],
          [-6.8393e-05, -7.8393e-05, -1.2604e-04,  ..., -1.2361e-04,
           -7.1149e-05, -4.1055e-05],
          ...,
   